In [41]:
import os
import numpy as np
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_absolute_error
import re

def evaluate_image_pairs(image_dir, mode='A'):
 
    assert mode in ['A', 'B'], "mode 参数必须是 'A' 或 'B'"

    target_fake = f'fake_{mode}'
    target_real = f'real_{mode}'
    label = f'{target_fake} vs {target_real}'

    # 正则匹配图像名并按 base 分组
    all_files = sorted(os.listdir(image_dir))
    pattern = r'(.*)_((fake|real)_[AB])\.png'
    grouped = {}

    for file in all_files:
        match = re.match(pattern, file)
        if match:
            base = match.group(1)
            tag = match.group(2)
            if base not in grouped:
                grouped[base] = {}
            grouped[base][tag] = file

    # 初始化结果
    results = {label: {"psnr": [], "ssim": [], "mae": []}}

    # 遍历并计算
    for base, files in grouped.items():
        if target_fake in files and target_real in files:
            fake_path = os.path.join(image_dir, files[target_fake])
            real_path = os.path.join(image_dir, files[target_real])

            img_fake = np.array(Image.open(fake_path)).astype(np.uint8)
            img_real = np.array(Image.open(real_path)).astype(np.uint8)
    
            
            if img_fake.shape != img_real.shape:
                print(f"⚠️ 尺寸不一致: {files[target_fake]} vs {files[target_real]}")
                continue

            psnr_val = psnr(img_fake, img_real, data_range=255)
            ssim_val = ssim(img_fake, img_real, data_range=255,channel_axis=-1)
            mae_val = mean_absolute_error(img_fake.flatten(), img_real.flatten())

            results[label]["psnr"].append(psnr_val)
            results[label]["ssim"].append(ssim_val)
            results[label]["mae"].append(mae_val)
    if results[label]["psnr"]:
        print(f"Avg PSNR: {np.mean(results[label]['psnr']):.2f}")
        print(f"Avg SSIM: {np.mean(results[label]['ssim']):.4f}")
        print(f"Avg MAE : {np.mean(results[label]['mae']):.4f}")
    else:
        print("⚠️ 没有匹配的图像对。")

    return results

In [43]:
cancer_type = ["KIRC", "LIHC"]
Direction = ["CT2MR", "MR2CT"]
Model = ["Resnet6", "Resnet9"]
base_dir = "C:/Users/fengs/Desktop"

for model in Model:
    for cancer in cancer_type:
        for direction in Direction:
            eventual_path = os.path.join(base_dir, f"Result({model})", f"{cancer}-{direction}")

            # 判断方向决定 A/B
            mode = "A" if direction.endswith("CT") else "B"

            print("Concrete Index Result Respect To Different Scenario")
            print(f"▶ Model: {model} | Cancer: {cancer} | Direction: {direction}")
            _ = evaluate_image_pairs(eventual_path, mode)


Concrete Index Result Respect To Different Scenario
▶ Model: Resnet6 | Cancer: KIRC | Direction: CT2MR
Avg PSNR: 14.04
Avg SSIM: 0.1808
Avg MAE : 35.2261
Concrete Index Result Respect To Different Scenario
▶ Model: Resnet6 | Cancer: KIRC | Direction: MR2CT
Avg PSNR: 14.47
Avg SSIM: 0.1982
Avg MAE : 32.9269
Concrete Index Result Respect To Different Scenario
▶ Model: Resnet6 | Cancer: LIHC | Direction: CT2MR
Avg PSNR: 14.84
Avg SSIM: 0.2716
Avg MAE : 31.1864
Concrete Index Result Respect To Different Scenario
▶ Model: Resnet6 | Cancer: LIHC | Direction: MR2CT
Avg PSNR: 14.54
Avg SSIM: 0.3178
Avg MAE : 32.2025
Concrete Index Result Respect To Different Scenario
▶ Model: Resnet9 | Cancer: KIRC | Direction: CT2MR
Avg PSNR: 14.82
Avg SSIM: 0.1873
Avg MAE : 32.4207
Concrete Index Result Respect To Different Scenario
▶ Model: Resnet9 | Cancer: KIRC | Direction: MR2CT
Avg PSNR: 13.06
Avg SSIM: 0.1798
Avg MAE : 40.1041
Concrete Index Result Respect To Different Scenario
▶ Model: Resnet9 | Cance